In [0]:
%sql
drop table if exists learning_spark_sql.varanasi_schema.varanasi_bronze;

In [0]:
%python
df_csv = (spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/Volumes/learning_spark_sql/varanasi_schema/raw_files_varanasi/varanasi_raw.csv")
)

# Now, write the DataFrame to Delta
df_csv.write.format("delta").mode("overwrite").saveAsTable(
    "learning_spark_sql.varanasi_schema.varanasi_bronze"
)

In [0]:
%sql 
select * from learning_spark_sql.varanasi_schema.varanasi_bronze;

In [0]:
%sql
SELECT School_Name
FROM (
  SELECT
    School_Name,
    ROW_NUMBER() OVER (ORDER BY School_Name) AS idx
  FROM learning_spark_sql.varanasi_schema.varanasi_bronze
) t
WHERE idx BETWEEN 21 AND 40;


In [0]:
%sql
SELECT School_Name
FROM (
  SELECT
    School_Name,
    ROW_NUMBER() OVER (ORDER BY School_Name) AS idx
  FROM learning_spark_sql.varanasi_schema.varanasi_bronze
) t
WHERE idx BETWEEN 41 AND 50;

In [0]:
%python
df_csv1 = (spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/Volumes/learning_spark_sql/varanasi_schema/raw_files_varanasi/top20varanasi.csv")
)
df_csv2 = (spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/Volumes/learning_spark_sql/varanasi_schema/raw_files_varanasi/21to40schools.csv")
)
df_csv3 = (spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/Volumes/learning_spark_sql/varanasi_schema/raw_files_varanasi/41to50schools.csv")
)

# Now, write the DataFrame to Delta
df_csv1.write.format("delta").mode("overwrite").saveAsTable(
    "learning_spark_sql.varanasi_schema.varanasi_bronze_top20"
)

In [0]:
df_csv2.write.format("delta").mode("overwrite").saveAsTable(
    "learning_spark_sql.varanasi_schema.varanasi_bronze_21_40"
)

df_csv3.write.format("delta").mode("overwrite").saveAsTable(
    "learning_spark_sql.varanasi_schema.varanasi_bronze_41_50"
)

In [0]:
%sql
select * from learning_spark_sql.varanasi_schema.varanasi_bronze_21_40;

In [0]:
%sql
-- Update from top20
MERGE INTO learning_spark_sql.varanasi_schema.varanasi_bronze AS target
USING learning_spark_sql.varanasi_schema.varanasi_bronze_top20 AS source
ON target.School_Name = source.School_Name
WHEN MATCHED THEN
  UPDATE SET target.Google_Maps_Link = source.Google_Maps_Link;


In [0]:
%sql
-- Update from top20
MERGE INTO learning_spark_sql.varanasi_schema.varanasi_bronze AS target
USING learning_spark_sql.varanasi_schema.varanasi_bronze_21_40 AS source
ON target.School_Name = source.School_Name
WHEN MATCHED THEN
  UPDATE SET target.Google_Maps_Link = source.Google_Maps_Link;


In [0]:
%sql

MERGE INTO learning_spark_sql.varanasi_schema.varanasi_bronze AS target
USING learning_spark_sql.varanasi_schema.varanasi_bronze_41_50 AS source
ON target.School_Name = source.School_Name
WHEN MATCHED THEN
  UPDATE SET target.Google_Maps_Link = source.Google_Maps_Link;

In [0]:
%sql
select * from learning_spark_sql.varanasi_schema.varanasi_bronze;

In [0]:
%sql
CREATE OR REPLACE TABLE learning_spark_sql.varanasi_schema.varanasi_bronze AS
SELECT
  ROW_NUMBER() OVER (ORDER BY School_Name) AS idx,
  *
FROM learning_spark_sql.varanasi_schema.varanasi_bronze;


In [0]:
%sql
select * from learning_spark_sql.varanasi_schema.varanasi_bronze;

In [0]:
%sql
drop table if exists learning_spark_sql.varanasi_schema.varanasi_bronze_indexed;

In [0]:
%sql

ALTER TABLE learning_spark_sql.varanasi_schema.varanasi_bronze
SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name');

ALTER TABLE learning_spark_sql.varanasi_schema.varanasi_bronze
DROP COLUMN `#`;


In [0]:
%sql
select * from learning_spark_sql.varanasi_schema.varanasi_bronze;

In [0]:
%python
# PySpark
df = spark.table("learning_spark_sql.varanasi_schema.varanasi_bronze")
# Small table → coalesce to 1 file
df.coalesce(1).write.option("header", True).mode("overwrite").csv("learning_spark_sql.varanasi_schema.bronze_to_csv_school")
